# FNOL (First Notice of Loss) Processing

## Overview

This notebook processes First Notice of Loss (FNOL) data for insurance claims. It extracts, validates, and structures FNOL data with completeness checks to prepare it for downstream processing by fraud detection and policy verification teams.

## Workflow

1. **Import Dependencies** - Load required libraries (Strands, AWS Bedrock, JSON)
2. **AWS Configuration** - Configure Bedrock client and model
3. **Load FNOL Data** - Read and parse FNOL.json file
4. **FNOL Processing Agent** - Extract, validate, and structure claim data
5. **Data Quality Checks** - Flag missing fields and inconsistencies
6. **Output Structured Data** - Generate validated JSON for next stages


## 1. Install Dependencies

In [ ]:
!pip install -r requirements.txt

## 2. Import Required Libraries

In [ ]:
# Standard library imports
import json
from datetime import datetime

# AWS SDK
import boto3

# Strands Agents SDK
from strands import Agent
from strands.models import BedrockModel

print("✓ All dependencies imported successfully")

## 3. AWS Bedrock Configuration

In [ ]:
# AWS Configuration
AWS_REGION = "us-east-1"
MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"

# Initialize Bedrock client to verify connectivity
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)

print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
print(f"✓ Using model: {MODEL_ID}")

## 4. Load FNOL Data

In [ ]:
# Load FNOL JSON data
with open('data/FNOL.json', 'r') as f:
    fnol_data = json.load(f)

print("✓ FNOL data loaded successfully")
print(f"\nClaim Number: {fnol_data['fnol']['claimNumber']}")
print(f"Report Date: {fnol_data['fnol']['reportDate']}")
print(f"Policyholder: {fnol_data['policyholder']['firstName']} {fnol_data['policyholder']['lastName']}")
print(f"Vehicle: {fnol_data['vehicle']['insuredVehicle']['year']} {fnol_data['vehicle']['insuredVehicle']['make']} {fnol_data['vehicle']['insuredVehicle']['model']}")

## 5. Initialize Bedrock Model

In [ ]:
# Initialize Bedrock model
model = BedrockModel(
    model_id=MODEL_ID,
    region_name=AWS_REGION
)

print("✓ Bedrock model initialized")

## 6. Create FNOL Processing Agent

This agent acts as a Claims Data Extraction Specialist that:
- Extracts key data points from FNOL documents
- Validates data completeness and consistency
- Flags potential data quality issues
- Generates structured JSON output for downstream processing

In [ ]:
# Create FNOL Processing Agent
fnol_agent = Agent(
    name="FNOL Data Extraction Specialist",
    model=model,
    system_prompt="""You are a Claims Data Extraction Specialist.

ROLE: Process First Notice of Loss (FNOL) forms and extract structured claim data.

INPUTS: Raw FNOL document/form data
OUTPUTS: Structured JSON with validated claim information

INSTRUCTIONS:
1. Extract key data points:
   - Policy number, claim date, incident location, claimant details
   - Incident description, damages reported, witnesses
   - Supporting documentation references

2. Validate data completeness:
   - Flag missing required fields
   - Identify inconsistencies in dates/locations
   - Check format compliance (policy numbers, contact info)

3. Flag potential data quality issues with specific error codes
4. If critical information is missing, generate specific follow-up questions

ERROR HANDLING: If form is illegible or severely incomplete, flag for manual review with detailed reasoning.

QUALITY CHECK: Ensure all extracted monetary amounts, dates, and identifiers are properly formatted.

OUTPUT FORMAT: Return a structured JSON object with the following sections:
- extraction_metadata: Processing date, data quality score, completeness status
- claim_information: Claim number, report date, policy number
- policyholder_information: Name, contact details, driver license
- incident_details: Date, time, location, description, weather conditions
- vehicle_information: Insured vehicle details, damage assessment
- other_parties: Information about other drivers/vehicles involved
- injuries: Details of any injuries reported
- validation_results: List of any missing fields, inconsistencies, or data quality issues
- recommendations: Next steps or follow-up actions needed
"""
)

print("FNOL Processing Agent created")

## 7. Process FNOL Data

In [ ]:
# Create processing prompt
processing_prompt = f"""
Please process the following FNOL (First Notice of Loss) data and extract all relevant information.

Perform the following tasks:
1. Extract and structure all key data points
2. Validate data completeness and identify any missing required fields
3. Check for inconsistencies in dates, locations, or other data
4. Assess data quality and flag any issues
5. Generate recommendations for next steps

FNOL Data:
{json.dumps(fnol_data, indent=2)}

Please provide a comprehensive structured JSON output with your analysis.
"""

print("=" * 80)
print("PROCESSING FNOL DATA")
print("=" * 80)
print("\n Agent is processing FNOL data...\n")

In [ ]:
# Run the agent
response = fnol_agent(processing_prompt)

print("\n" + "=" * 80)
print("✓ FNOL Processing completed successfully")
print("=" * 80)

## 8. Display Processing Results

In [ ]:
# Display agent's response
print("=" * 80)
print("FNOL PROCESSING RESULTS")
print("=" * 80)
print()
print(response)
print()
print("=" * 80)